## Data collection for cancer drug response prediction project
This notebook illustrates the data collecting step for the project. Reletaed raw data will be downloaded from websites and then processed into data matrix which will be used as input for the machine learning models.
### Downloading raw data from GDSC and COSMIC
Genomics of Drug Sensitivity in Cancer (GDSC) provide cancer drug response data over 1001 COSMIC cell lines, which contains Natural log half maximal inhibitory concentration (IC50) and Area under the dose-response curve (AUCs) values for all screened cell line/drug combinations. The list of screened compounds and the corresponding response data can be downloaded from the GDSC website (https://www.cancerrxgene.org/downloads). The cell lines invovled are from COSMIC Cell Lines Project, and the complete cell line profiles such as mutation data, raw gene expression data, and copy number data can be downloaded from the COSMIC website (https://cancer.sanger.ac.uk/cell_lines/download).

### Data processing
#### Import packages
After downloading data from GDSC and COSMIC, we will use Pandas and Numpy packages for data processing.  
Pandas: https://pandas.pydata.org/  
Numpy: http://www.numpy.org/

In [1]:
import pandas as pd
import numpy as np

#### Read cell line information
First we need to read the csv file containing cell line details. It lists out the whole set of cell lines used in this project, their identifiers, and what kind of data is included in this study. The COSMIC_ID will be used as the index for our data matrix. The example five rows are listed below.

In [3]:
cell_lines = pd.read_csv('../data/Cell_Lines_Details.csv', dtype={'COSMIC identifier': int}) 
cell_lines.rename(columns={'COSMIC identifier':'COSMIC_ID'}, inplace = True)
cell_lines.head()

,Sample Name,COSMIC_ID,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug Response,GDSC Tissue descriptor 1,GDSC Tissue descriptor 2,Cancer Type (matching TCGA label),Microsatellite instability Status (MSI),Screen Medium,Growth Properties
0,A253,906794,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,NaN,MSS/MSI-L,D/F12,Adherent
1,BB30-HNC,753531,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
2,BB49-HNC,753532,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
3,BHY,753535,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
4,BICR10,1290724,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent


#### Read mutation data and build mutation matrix
The whole set of mutation profiles is in the file *CosmicCLP_MutantExport.tsv* (as the raw file is too large to upload, you can download it from https://cancer.sanger.ac.uk/cell_lines/download), the coding silent mutation will be removed from the input data, and the format of this file is shown in the example five rows.

In [5]:
mutation = pd.read_csv('../data/CosmicCLP_MutantExport.tsv', sep='\t', dtype='unicode') 
mutation = mutation.loc[mutation['Mutation Description'] != 'Substitution - coding silent']  # remove coding silent mutation
mutation['Gene name'] = mutation['Gene name'].replace(to_replace=r'_.*', value='', regex=True) # formatting gene names
mutation['Gene name'] = mutation['Gene name'].str.upper()
mutation.head()

,Gene name,Accession Number,Gene CDS length,HGNC ID,Sample name,ID_sample,ID_tumour,Primary site,Site subtype 1,Site subtype 2,...,Mutation somatic status,Mutation verification status,Pubmed_PMID,ID_STUDY,Institute,Institute Address,Catalogue Number,Sample source,Tumour origin,Age
0,KRAS,ENST00000311936,567,6407,PL-21,1330991,1241467,haematopoietic_and_lymphoid_tissue,NS,NS,...,Reported in another cancer sample as somatic,Verified,NaN,619,German Collection of Microorganisms and Cell C...,"Braunschweig, Germany",ACC 536,cell-line,NS,NaN
1,P2RY2,ENST00000393596,1134,8541,A375,906793,824317,skin,NS,NS,...,Reported in another cancer sample as somatic,Unverified,NaN,619,American Type Culture Collection (ATCC),"P.O. Box 1549, Manassas, VA 20108, USA",CRL-1619,cell-line,primary,54
2,SALL4,ENST00000217086,3162,15924,MCC26,1298234,1209288,skin,NS,NS,...,Variant of unknown origin,Unverified,NaN,619,UNKNOWN,UNKNOWN,NaN,cell-line,NS,NaN
3,TP53,ENST00000413465,858,11998,OV-7,1480360,1404019,ovary,NS,NS,...,Reported in another cancer sample as somatic,Unverified,NaN,619,Cancer Science Institute of Singapore,National University of Singapore,NaN,cell-line,NS,NaN
5,COL14A1,ENST00000297848,5391,2191,RH-1,971773,887870,soft_tissue,striated_muscle,NS,...,Reported in another cancer sample as somatic,Unverified,NaN,619,St Jude Children's Research Hospital,"332 North Lauderdale St., Memphis, TN 38105-27...",NaN,cell-line,metastasis,NaN


In [6]:
genes_mut = list(set(mutation['Gene name']))
print("The number of gene with mutation status is " + str(len(genes_mut)))

The number of gene with mutation status is 19392


Create a new data matrix to store the mutation status for each cell lines. Each row is the profile of one cell line, and each raw is one gene. If the gene is mutated in a cell line, then the corresponding position in the matrix would be 1 and 0 for wild type. 

In [7]:
mut_df = pd.DataFrame(index=cell_lines['COSMIC_ID'], columns=genes_mut)
mut_df = mut_df.fillna(0)
gene_name = list(mutation['Gene name'])
id_sample = list(mutation['ID_sample'])
for i in range(len(mutation)):
    if int(id_sample[i]) in mut_df.index:
        mut_df.at[int(id_sample[i]), gene_name[i]] = 1

Output mutation matrix into csv file, which could be used in model training.

In [9]:
mut_df.to_csv("../data/mutation_matrix.csv")

#### Read gene expression data and build expression matrix
The whole set of raw expression profiles (with raw expression values) is in the file *CosmicCLP_RawGeneExpression.tsv* (as the raw file is too large to upload, you can download it from https://cancer.sanger.ac.uk/cell_lines/download), and the format of this file is shown in the example five rows. Here the SAMPLE_ID is corresponding to the COSMIC_ID of the cell line list. (You could also use processed expression data, which is *CosmicCLP_CompleteGeneExpression.tsv* if you want)

In [11]:
exp = pd.read_csv('../data/CosmicCLP_RawGeneExpression.tsv', sep='\t', dtype='unicode') 
exp['GENE_NAME'] = exp['GENE_NAME'].replace(to_replace=r'_.*', value='', regex=True)
exp['GENE_NAME'] = exp['GENE_NAME'].str.upper()
exp.head()

,SAMPLE_ID,SAMPLE_NAME,GENE_NAME,GENE_EXPRESSION
0,683665,MC-CAR,A1BG,-0.430
1,683665,MC-CAR,A1CF,-0.860
2,683665,MC-CAR,A2M,1.070
3,683665,MC-CAR,A2ML1,0.420
4,683665,MC-CAR,A3GALT2P,2.740


In [12]:
genes_exp = list(set(exp['GENE_NAME']))
print("The number of gene with expression profile is " + str(len(genes_exp)))

The number of gene with expression profile is 16374


Create a new data matrix to store the expression values. Each row represents the expression profile of one cell line, and each raw is one gene.

In [15]:
exp_df = pd.DataFrame(index=cell_lines['COSMIC_ID'], columns=genes_exp)
gene_name = list(exp['GENE_NAME'])
id_sample = list(exp['SAMPLE_ID'])
n = len(exp)
for i in range(n):
    if i % 2000000 == 0 and i > 0:
        print(str(i) + "/" + str(n) + " processed.")
    if int(id_sample[i]) in exp_df.index:
        exp_df.at[int(id_sample[i]), gene_name[i]] = exp.iloc[i, 3]

2000000/15866406 processed.
4000000/15866406 processed.
6000000/15866406 processed.
8000000/15866406 processed.
10000000/15866406 processed.
12000000/15866406 processed.
14000000/15866406 processed.


Output expression matrix into csv file, which could be used in model training.

In [16]:
exp_df.to_csv("../data/raw_expression_matrix.csv")

#### Read copy number data and build copy number matrix
The whole set of copy number variation information is in the file *CosmicCLP_CompleteCNA.tsv* (as the raw file is too large to upload, you can download it from https://cancer.sanger.ac.uk/cell_lines/download), and the format of this file is shown in the example five rows. Here the ID_SAMPLE is corresponding to the COSMIC_ID of the cell line list.

In [17]:
cnv = pd.read_csv('../data/CosmicCLP_CompleteCNA.tsv', sep='\t', dtype='unicode') 
cnv['gene_name'] = cnv['gene_name'].replace(to_replace=r'_.*', value='', regex=True)     # formatting the gene name as above
cnv['gene_name'] = cnv['gene_name'].str.upper()
cnv.head()

,CNV_ID,ID_GENE,gene_name,ID_SAMPLE,ID_TUMOUR,Primary site,Site subtype 1,Site subtype 2,Site subtype 3,Primary histology,Histology subtype 1,Histology subtype 2,Histology subtype 3,SAMPLE_NAME,TOTAL_CN,MINOR_ALLELE,MUT_TYPE,ID_STUDY,GRCh,Chromosome:G_Start..G_Stop
0,6369721,101525,GYG2P1,683665,611825,haematopoietic_and_lymphoid_tissue,NS,NS,NS,lymphoid_neoplasm,plasma_cell_myeloma,NS,NS,MC-CAR,5,0,gain,619,38,Y:12363966..12512040
1,6619186,69785,DAZ2,683665,611825,haematopoietic_and_lymphoid_tissue,NS,NS,NS,lymphoid_neoplasm,plasma_cell_myeloma,NS,NS,MC-CAR,7,0,gain,619,38,Y:22477961..24258193
2,6371065,68758,CTDSPL,683665,611825,haematopoietic_and_lymphoid_tissue,NS,NS,NS,lymphoid_neoplasm,plasma_cell_myeloma,NS,NS,MC-CAR,0,0,loss,619,38,3:37940617..37945438
3,6388134,55218,LCE3C,683665,611825,haematopoietic_and_lymphoid_tissue,NS,NS,NS,lymphoid_neoplasm,plasma_cell_myeloma,NS,NS,MC-CAR,0,0,loss,619,38,1:152583052..152613763
4,6619186,69786,CDY1B,683665,611825,haematopoietic_and_lymphoid_tissue,NS,NS,NS,lymphoid_neoplasm,plasma_cell_myeloma,NS,NS,MC-CAR,7,0,gain,619,38,Y:22477961..24258193


In [18]:
genes_cnv = list(set(cnv['gene_name']))
print("The number of gene with copy number variation profile is " + str(len(genes_cnv)))

The number of gene with copy number variation profile is 17614


In [19]:
cnv_df = pd.DataFrame(index=cell_lines['COSMIC_ID'], columns=genes_cnv)
cnv_df = cnv_df.fillna(0)
gene_name = list(cnv['gene_name'])
id_sample = list(cnv['ID_SAMPLE'])
cnv_status = list(cnv['MUT_TYPE'])
for i in range(len(cnv)):
    if int(id_sample[i]) in cnv_df.index:
        cnv_df.at[int(id_sample[i]), gene_name[i]] = cnv_status[i]

Output CNV matrix into csv file, which could be used in model training.

In [20]:
cnv_df.to_csv("../data/cnv_matrix.csv")

### Generating drug fingerprint matrix for the deep learning model
The drug response data only contain IC50 values over several hundred examples(cell lines) for each of the drugs. Then if we build one predicting model for each drug, then we only have hundreds of training examples, which is not enough for the deep learning model. So we changed to input drug as features and build only one comprehensive model, which will be feeded genomic profiles and the drug fingerprints(features) as input and predict the IC50 values. Here, we will build the drug matrix containing features for each drug. 

#### Read drug list

In [27]:
drug_list = pd.read_csv('../data/Screened_Compounds.csv', dtype='unicode')
drug_list = drug_list.drop(['SYNONYMS', 'TARGET', 'TARGET_PATHWAY'], axis=1)
print(drug_list.head())
print('Total number of drug is ' + str(len(list(drug_list['DRUG_ID']))))

  DRUG_ID   DRUG_NAME
0       1   Erlotinib
1       3   Rapamycin
2       5   Sunitinib
3       6  PHA-665752
4       9      MG-132
Total number of drug is 267


#### Download drug structure  information from PubChem
PubChem is a database of chemical molecules provided by NCBI (https://pubchem.ncbi.nlm.nih.gov/). We will download the SMILES format representation of the drug compounds from this database. For some of the compounds that we can not find in this database, we will manually download from LINCS database (http://lincs.hms.harvard.edu/db/sm/). If we still can not find the structures from both databases, we will remove the drug compounds from our list.  
  
A python package named Pubchempy (http://pubchempy.readthedocs.io/en/latest/) will be used to retrive SMILES representation from PubChem.

In [22]:
import pubchempy as pcp
smiles = []
weights = []
for drug in list(drug_list['DRUG_NAME']):
    results = pcp.get_compounds(drug, 'name')
    print(str(drug) + " " + str(results))
    if results == []:
        smiles.append('')
        weights.append(0)
    else:
    #for compound in results:
        smiles.append(results[0].isomeric_smiles)
        weights.append(results[0].molecular_weight)

Store drug SMILES information in a csv file.

In [ ]:
drugs = pd.DataFrame(index=list(drug_list['DRUG_NAME']), columns=['SMILES', 'Weight'])
for i in range(len(drug_list)):
    drugs.loc[drug_list[i], 'SMILES'] = smiles[i]
    drugs.loc[drug_list[i], 'Weight'] = weights[i]
drugs.to_csv('drug_smiles.csv')

Write the SMILES of each drugs to an individual file(.smi). The compounds with molecular weight > 1000 will be removed. These files will be used as the input to the PaDEL-Descriptor software in order to generate fingerprints. 

In [30]:
#drugs = pd.read_csv('D:/SNP/drug_smiles.csv') 
for i in range(251):
    if pd.isnull(drugs.iloc[i,1]):
        continue
    drug_name = drugs.iloc[i,0]
    if float(drugs.iloc[i,2]) > 1000:
        print(drug_name + ' molecular_weight larger than 1000')
        continue
    if drug_name == 'VNLG/124':
        drug_name = 'VNLG-124'
    f = open("../data/smiles/" + drug_name + ".smi", 'w')
    f.write(drugs.iloc[i,1] + ' ' + drugs.iloc[i,0])
    f.close()

#### Generate fingerprint with PaDEL-Descriptor
PaDEL-Descriptor is a software to calculate molecular descriptors and fingerprints (http://www.yapcwsoft.com/dd/padeldescriptor/). You could find the instructions of how to use this software on the website. Here 
we will use it to extract descriptors of three classes of fingerprints: (1) fingerprinter, (2) extended fingerprinter, and (3) graph only finger-printer. There are 3072 binary descriptors for each drug, which will be used as drug features in our model. So each drug will be represented as a feature vector with 3072 binary features.  
The resulting matrix is in the file *drug_fingerprint_complete.csv*